In [ ]:
!pip install -q --upgrade accelerate peft bitsandbytes transformers trl flash_attn
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

In [ ]:
BASE_PATH="bone-fracture-dataset/Bone_Fracture_Dataset/"
folders={"dataset/test/images/":"dataset/test_new/images/","dataset/train/images/":"dataset/new_train/images/"}

In [ ]:
# Step 1: Upload kaggle.json
from google.colab import files
files.upload()

# Step 2: Configure Kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 3: Download Dataset
!kaggle datasets download -d tuantm04/bone-fracture-dataset

# Step 4: Unzip Dataset
!unzip bone-fracture-dataset.zip -d bone-fracture-dataset

# Step 5: Verify Files
!ls bone-fracture-dataset


Streaming output truncated to the last 5000 lines.
  inflating: bone-fracture-dataset/Bone_Fracture_Dataset/dataset/train/labels/IMG0000367.txt  
  inflating: bone-fracture-dataset/Bone_Fracture_Dataset/dataset/train/labels/IMG0000370.txt  
  inflating: bone-fracture-dataset/Bone_Fracture_Dataset/dataset/train/labels/IMG0000371.txt  
  inflating: bone-fracture-dataset/Bone_Fracture_Dataset/dataset/train/labels/IMG0000372.txt  
  inflating: bone-fracture-dataset/Bone_Fracture_Dataset/dataset/train/labels/IMG0000373.txt  
  inflating: bone-fracture-dataset/Bone_Fracture_Dataset/dataset/train/labels/IMG0000374.txt  
  inflating: bone-fracture-dataset/Bone_Fracture_Dataset/dataset/train/labels/IMG0000375.txt  
  inflating: bone-fracture-dataset/Bone_Fracture_Dataset/dataset/train/labels/IMG0000376.txt  
  inflating: bone-fracture-dataset/Bone_Fracture_Dataset/dataset/train/labels/IMG0000377.txt  
  inflating: bone-fracture-dataset/Bone_Fracture_Dataset/dataset/train/labels/IMG0000378.txt  

In [ ]:
from google.colab import auth
import os
import json
from PIL import Image
import io
import torch
from google.cloud import storage

from transformers import PaliGemmaForConditionalGeneration
from transformers import BitsAndBytesConfig
from peft import get_peft_model, LoraConfig
from google.colab import userdata

In [ ]:
auth.authenticate_user()

In [ ]:
import os
os.environ["HF_TOKEN"] = "" #add HF token
PROJECT_ID = "favorable-order-437011-k2" ## add your project ID
BUCKET_NAME = "finetunetesting" ## add Bucket name
DATA_FOLDER = "dataset/" # provide folder path inside the bucket where the images are present

In [ ]:
def denormalize_coordinates(norm_x, norm_y, norm_width, norm_height, image_width, image_height):
    # Convert center coordinates to pixels
    x = int(norm_x * image_width)
    y = int(norm_y * image_height)
    width = int(norm_width * image_width)
    height = int(norm_height * image_height)

    # Calculate corner points
    x1 = int(x - width/2)
    y1 = int(y - height/2)
    x2 = int(x + width/2)
    y2 = int(y + height/2)

    return x1, y1, x2, y2


In [ ]:
import cv2
import pandas as pd
def create_bounding_box(csv_file, path, new_path):
    # Create the new path if it doesn't exist
    if not os.path.exists(new_path + "images/"):
        os.makedirs(new_path + "images/")

    df = pd.read_csv(csv_file, header=None)
    df.head()

    for index, row in df.iterrows():
        # Read the image
        image = cv2.imread(path + "images/" + row[0])
        if image is None:
            print(f"Image {row[0]} not found, skipping.")
            continue

        height, width, channels = image.shape

        # Read the label file
        label_file_path = path + "labels/" + row[1]
        if not os.path.exists(label_file_path):
            print(f"Label file {row[1]} not found, skipping.")
            continue

        with open(label_file_path) as f:
            labels = f.read()
            for label in labels.split("\n"):
                if label:
                    coordinates = label.split(" ")
                    x, y, width, height = denormalize_coordinates(
                        float(coordinates[1]),
                        float(coordinates[2]),
                        float(coordinates[3]),
                        float(coordinates[4]),
                        width,
                        height
                    )
                    start_point = (int(x), int(y))
                    end_point = (int(width), int(height))
                    color = (0, 255, 0)  # Green color in BGR
                    thickness = 2
                    # Draw rectangle on the image
                    image = cv2.rectangle(image, start_point, end_point, color, thickness)

        # Save the modified image to the new path
        cv2.imwrite(new_path + "images/" + row[0], image)


create_bounding_box(BASE_PATH+"train.csv",BASE_PATH+"dataset/train/",BASE_PATH+"dataset/new_train/")
create_bounding_box(BASE_PATH+"test.csv",BASE_PATH+"dataset/test/",BASE_PATH+"dataset/new_test/")

In [ ]:
import json
import pandas as pd
df=pd.read_csv(BASE_PATH+"train.csv",header=None)
df.head()
for index, row in df.iterrows():
    data = {
        "image": "dataset/train/images/"+row[0],
        "data": "Image of hand x-ray"
    }


    with open(BASE_PATH + "train.jsonl", "a") as f:
        json.dump(data, f)
        f.write('\n')

In [ ]:
from transformers import AutoProcessor
model_id = "microsoft/Phi-3.5-vision-instruct"
processor = AutoProcessor.from_pretrained(model_id,
  trust_remote_code=True,
  num_crops=4
)

processor_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

processing_phi3_v.py:   0%|          | 0.00/22.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-vision-instruct:
- processing_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:520: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/442 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_token = processor.tokenizer.convert_tokens_to_ids("<image>")


In [ ]:
from transformers import AutoModelForCausalLM
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="float16",
        bnb_4bit_use_double_quant=True,
    )


lora_config = LoraConfig(
    r=1,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj",
                    "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)


model = AutoModelForCausalLM.from_pretrained(
  model_id,
  device_map="cuda",
  trust_remote_code=True,
  _attn_implementation='eager',
  quantization_config=bnb_config
).to('cuda')

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

config.json:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

configuration_phi3_v.py:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-vision-instruct:
- configuration_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3_v.py:   0%|          | 0.00/88.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-vision-instruct:
- modeling_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
# Load model directly
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  device_map="cuda",
  trust_remote_code=True,
  _attn_implementation='eager',
  quantization_config=bnb_config
).to('cuda')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import TrainingArguments
args=TrainingArguments(
            num_train_epochs=1,
            remove_unused_columns=False,
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            warmup_steps=2,
            learning_rate=2e-5,
            weight_decay=1e-6,
            adam_beta2=0.999,
            logging_steps=1,
            optim="adamw_hf",
            save_strategy="steps",
            save_steps=1000,
            push_to_hub=False,
            save_total_limit=1,
            output_dir="paligemma_vqav2",
            bf16=True,
            report_to=["tensorboard"],
            dataloader_pin_memory=False,
        )


In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 704,512 || all params: 4,147,325,952 || trainable%: 0.0170


In [ ]:

from PIL import Image
import os

def get_image_from_local(image_file_name):
    image_path = os.path.join(BASE_PATH, image_file_name)

    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file {image_path} does not exist.")

    image = Image.open(image_path)
    return image


In [ ]:
def collate_fn(examples):
  texts = ["answer " + "Are there are any abnormalities in the xray "]
  labels = [example['data'] for example in examples]
  images = [get_image_from_local(example["image"]).convert("RGB") for example in examples]
  tokens = processor(texts, images, return_tensors="pt")

  tokens = tokens.to(texts.bfloat16).to(device)
  return tokens

def collate_fn(examples):
    # Extract image and labels
    images = [get_image_from_local(example["image"]).convert("RGB") for example in examples]
    labels = [example["data"] for example in examples]

    # Define user prompt
    user_prompt = "Are there any abnormalities in the x-ray?"
    example = examples[0]

    # Generate message template for the user prompt
    messages = [
        {"role": "user", "content": f"<|image_1|>\n{user_prompt}"}
    ]

    # Apply processor template and generate tokens for the prompt
    prompt = processor.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    # Combine labels into the expected answer format
    answer = f"{labels[0]}<|end|>\n<|endoftext|>"

    # Process prompt and image for tokenization
    batch = processor(prompt, images, return_tensors="pt")
    prompt_input_ids = batch["input_ids"]

    # Tokenize the answer for label preparation
    answer_input_ids = processor.tokenizer(answer, add_special_tokens=False, return_tensors="pt")["input_ids"]

    # Concatenate prompt and answer input IDs
    concatenated_input_ids = torch.cat([prompt_input_ids, answer_input_ids], dim=1)

    # Create labels with ignore index for prompt tokens
    ignore_index = -100
    labels = torch.cat(
        [
            torch.tensor([ignore_index] * len(prompt_input_ids[0])).unsqueeze(0),
            answer_input_ids,
        ],
        dim=1,
    )

    # Update batch with concatenated inputs and labels
    batch["input_ids"] = concatenated_input_ids
    batch["labels"] = labels

    # Ensure only floating-point tensors require gradients
    for key, value in batch.items():
        if isinstance(value, torch.Tensor) and torch.is_floating_point(value):
            batch[key] = value.clone().detach().requires_grad_(True)

    return batch


In [ ]:

def load_jsonl(filepath):
  """
  Loads a JSONL file into a list of dictionaries.

  Args:
    filepath: Path to the JSONL file.

  Returns:
    A list of dictionaries.
  """
  examples = []
  max_lines=2000
  with open(filepath, "r") as f:
    for i, line in enumerate(f):
      if max_lines is not None and i >= max_lines:
        break
      examples.append(json.loads(line))
  return examples


In [ ]:
filepath = BASE_PATH+"train.jsonl"
examples = load_jsonl(filepath)

In [ ]:
from transformers import Trainer

trainer = Trainer(
        model=model,
        train_dataset=examples ,
        data_collator=collate_fn,
        args=args
        )


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RuntimeError: FlashAttention only supports Ampere GPUs or newer.

In [ ]:
# prompt: delete non empty direct from curent folder name bone-frature-dataset

import shutil
import os
# Specify the directory to delete
directory_to_delete = "bone-fracture-dataset"  # Corrected directory name

try:
    # Check if the directory exists
    if os.path.exists(directory_to_delete):
        # Check if the directory is not empty
        if os.listdir(directory_to_delete):
          # Delete the directory and its contents
          shutil.rmtree(directory_to_delete)
          print(f"Directory '{directory_to_delete}' and its contents have been deleted.")
        else:
          print(f"Directory '{directory_to_delete}' is empty. Nothing to delete.")
    else:
        print(f"Directory '{directory_to_delete}' does not exist.")

except OSError as e:
    print(f"Error deleting directory: {e}")

Directory 'bone-fracture-dataset' and its contents have been deleted.
